In [51]:
import cv2
import numpy as np
import os 
import face_recognition
import matplotlib.pyplot as plt
from datetime import datetime  

In [52]:
path= r'F:\project_face'
images = []
classNames = []
personsList = os.listdir(path)
personsList

['AHMED.jpg', 'Al-Huwaini.jpg', 'El Shaarawy.jpg', 'Omar.jpg']

In [53]:
#load images
for cl in personsList:
    curPersonn = cv2.imread(f'{path}/{cl}')
    images.append(curPersonn)
    classNames.append(os.path.splitext(cl)[0])

In [54]:
def markAttendance(name):
    with open('list1.csv','r+') as f:
        myDataList = f.readlines()
        nameList=[]
        for line in myDataList:
            entry=line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now=datetime.now()
            dstring=now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dstring}')

In [55]:
def findEncodeings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList


In [56]:
encodeListKnown = findEncodeings(images)
print('Encoding Complete.')

Encoding Complete.


In [60]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    _, img = cap.read()

    imgS = cv2.resize(img, (0,0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    faceCurentFrame = face_recognition.face_locations(imgS)
    encodeCurentFrame = face_recognition.face_encodings(imgS, faceCurentFrame)

    for encodeface, faceLoc in zip(encodeCurentFrame, faceCurentFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeface)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeface)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            print(name)
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0,0,255), 2)
            cv2.rectangle(img, (x1,y2-35), (x2,y2), (0,0,255), cv2.FILLED)
            cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2)
            markAttendance(name)
    cv2.imshow('Face Recogntion', img)
    if cv2.waitKey(1) & 0xff ==ord('q'):
        break

In [62]:
cap.release()
cv2.destroyAllWindows()